JParaCrawl

In [1]:
!wget http://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/2.0/bitext/en-ja.tar.gz

--2021-07-03 12:04:59--  http://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/2.0/bitext/en-ja.tar.gz
Resolving www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)... 129.60.39.163
Connecting to www.kecl.ntt.co.jp (www.kecl.ntt.co.jp)|129.60.39.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1294917545 (1.2G) [application/x-gzip]
Saving to: ‘en-ja.tar.gz’

en-ja.tar.gz        100%[===================>]   1.21G   226KB/s    in 46m 17s 

2021-07-03 12:51:16 (455 KB/s) - ‘en-ja.tar.gz’ saved [1294917545/1294917545]

en-ja/
en-ja/LICENSE
en-ja/en-ja.bicleaner05.txt


In [ ]:
import tarfile

with tarfile.open('en-ja.tar.gz') as tar:
    for f in tar.getmembers():
        if f.name.endswith('txt'):
            text = tar.extractfile(f).read().decode('utf-8')
            break

data = text.splitlines()
data = [x.split('\t') for x in data]
data = [x for x in data if len(x) == 4]
data = [[x[3], x[2]] for x in data]
with open('jparacrawl.ja', 'w') as f:
  with open('jparacrawl.en', 'w') as g:
    for j, e in data:
        print(j, file=f)
        print(e, file=g)

In [ ]:
!pip install subword-nmt
!subword-nmt apply-bpe -c kyoto_en.codes < jparacrawl.en > train.jparacrawl.en

KFTT

In [ ]:
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
!tar -zxvf kftt-data-1.0.tar.gz

In [ ]:
!mkdir data-bin
!mkdir data-bin/kftt.ja-en
!pip install sentencepiece
import sentencepiece as spm
import re
spm.SentencePieceTrainer.Train('--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0')

sp = spm.SentencePieceProcessor()
sp.Load('kyoto_ja.model')

for src, dst in [
    ('kftt-data-1.0/data/orig/kyoto-train.ja', 'bpe-kyoto-train.ja'),
    ('kftt-data-1.0/data/orig/kyoto-dev.ja', 'bpe-kyoto-dev.ja'),
    ('kftt-data-1.0/data/orig/kyoto-test.ja', 'bpe-kyoto-test.ja'),
]:
  with open(src) as f:
    with open(dst, 'w') as g:
          for x in f:
              x = x.strip()
              x = re.sub(r'\s+', ' ', x)
              x = sp.encode_as_pieces(x)
              x = ' '.join(x)
              print(x, file=g)

!pip install subword-nmt
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/tok/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-train.en > bpe-kyoto-train.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-dev.en > bpe-kyoto-dev.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/tok/kyoto-test.en > bpe-kyoto-test.en

In [ ]:
!pip3 install fairseq

In [ ]:
!pip install sentencepiece
import sentencepiece as spm
import re
sp = spm.SentencePieceProcessor()
with open('jparacrawl.ja') as f:
  with open('train.jparacrawl.ja', 'w') as g:
    for x in f:
        x = x.strip()
        x = re.sub(r'\s+', ' ', x)
        x = sp.encode_as_pieces(x)
        x = ' '.join(x)
        print(x, file=g)

学習

In [ ]:
!fairseq-preprocess \
  --source-lang ja --target-lang en \
  --trainpref train.jparacrawl \
  --validpref dev.sub \
  --destdir data98 \
  --thresholdsrc 5 \
  --thresholdtgt 5 \
  --workers 20

In [ ]:
!pip install sacremoses
!CUDA_VISIBLE_DEVICES=0 fairseq-train data98 \
  --fp16 \
  --save-dir save98_1 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.2 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock98_1.log

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data98_1 \
  --path save98_1/checkpoint_best.pt < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 98_1.out

In [ ]:
fairseq-score --sys 98_1.out --ref bpe-kyoto-test.en

KFTT学習

In [ ]:
!fairseq-preprocess \
  --source-lang ja --target-lang en \
  --trainpref bpe-kyoto-train \
  --validpref bpe-kyoto-dev \
  --testpref bpe-kyoto-test \
  --tgtdict data98/dict.en.txt \
  --srcdict data98/dict.ja.txt \
  --destdir data98_2 \
  --thresholdsrc 5 \
  --thresholdtgt 5 \
  --workers 20

In [ ]:
 !CUDA_VISIBLE_DEVICES=0 fairseq-train data98_2 \
  --fp16 \
  --restore-file save98_1/checkpoint1.pt \
  --save-dir save98_2 \
  --max-epoch 1 \
  --arch transformer --share-decoder-input-output-embed \
  --optimizer adam --clip-norm 1.0 \
  --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --update-freq 1 \
  --dropout 0.2 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-remove-bpe \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric > knock98_2.log

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-interactive data98_2 \
  --path save98_2/checkpoint_best.pt < bpe-kyoto-test.ja \
  | grep '^H' | cut -f3 | sed -r 's/(@@ )|(@@ ?$)//g' > 98_2.out

In [ ]:
fairseq-score --sys 98_2.out --ref bpe-kyoto-test.en